# PCA from scratch

Principal component analysis (PCA) is a dimensionality reduction technique that transforms a data set into a set of orthogonal components — called principal components — which capture the maximum variance in the data. PCA simplifies complex data sets while preserving their most important structures.

# Why it matters?

Let’s say you are trying to predict house prices and you have 10 columns, like square footage, number of rooms, distance from the city, and so on.

Some of these columns may be giving you the same kind of information. PCA finds patterns in these columns and builds new axes to represent the data more efficiently.

These new axes are called principal components. Each one shows how much of the total variation in the data it can explain.

This video does an excellent job of explaining PCA in very simple terms.

[PCA Visually Explained](https://youtu.be/FD4DeN81ODY?si=6wYW_4rYxHdPWpWr)

# How Principal Component Analysis Works

# Dataset

For the following walkthrough, we will be working with the famous “Iris” dataset that has been deposited on the UCI machine learning repository
(https://archive.ics.uci.edu/ml/datasets/Iris).

The iris dataset contains measurements for 150 iris flowers from three different species.

The three classes in the Iris dataset are:

Iris-setosa (n=50)
Iris-versicolor (n=50)
Iris-virginica (n=50)
And the four features of in Iris dataset are:

sepal length in cm
sepal width in cm
petal length in cm
petal width in cm


## Loading Dataset

In order to load the Iris data directly from the UCI repository, we are going to use the superb pandas library.

In [24]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
from itertools import combinations
from sklearn.preprocessing import StandardScaler


In [2]:


df = pd.read_csv(
    filepath_or_buffer='https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',
    header=None,
    sep=',')

df.columns=['sepal_len', 'sepal_wid', 'petal_len', 'petal_wid', 'class']
df.dropna(how="all", inplace=True) # drops the empty line at file-end

df.tail()

,sepal_len,sepal_wid,petal_len,petal_wid,class
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica
149,5.9,3.0,5.1,1.8,Iris-virginica


In [16]:
# split data table into data X and class labels y

X = df.iloc[:,0:4].values
y = df.iloc[:,4].values

In [9]:
# Feature columns and names
feature_cols = ['sepal_len', 'sepal_wid', 'petal_len', 'petal_wid']
feature_names = ['Sepal Length (cm)', 'Sepal Width (cm)', 'Petal Length (cm)', 'Petal Width (cm)']

# Color mapping for classes
color_discrete_map = {
    'Iris-setosa': '#FF6B6B',
    'Iris-versicolor': '#4ECDC4', 
    'Iris-virginica': '#45B7D1'
}

# 1. SCATTER PLOT MATRIX
print("Creating Scatter Plot Matrix...")

# Calculate number of unique pairs
n_features = len(feature_cols)
pairs = list(combinations(range(n_features), 2))
n_pairs = len(pairs)

# Create subplot grid (3x2 for 6 pairs)
rows = 2
cols = 3

fig_scatter = make_subplots(
    rows=rows, 
    cols=cols,
    # subplot_titles=[f"{feature_names[pair[1]]} vs {feature_names[pair[0]]}" for pair in pairs],
    vertical_spacing=0.15,
    horizontal_spacing=0.1
)

# Add scatter plots for each pair
for idx, (i, j) in enumerate(pairs):
    row = (idx // cols) + 1
    col = (idx % cols) + 1
    
    for class_name, color in color_discrete_map.items():
        class_data = df[df['class'] == class_name]
        
        fig_scatter.add_trace(
            go.Scatter(
                x=class_data[feature_cols[i]],
                y=class_data[feature_cols[j]],
                mode='markers',
                name=class_name,
                marker=dict(
                    color=color,
                    size=8,
                    opacity=0.7,
                    line=dict(width=1, color='white')
                ),
                showlegend=(idx == 0),  # Only show legend once
                legendgroup=class_name
            ),
            row=row, col=col
        )
    
    # Update axes for this subplot
    fig_scatter.update_xaxes(
        title_text=feature_names[i], 
        row=row, col=col,
        showgrid=True,
        gridcolor='lightgray'
    )
    fig_scatter.update_yaxes(
        title_text=feature_names[j], 
        row=row, col=col,
        showgrid=True,
        gridcolor='lightgray'
    )

# Update scatter plot layout
fig_scatter.update_layout(
    title={
        'text': "Iris Dataset - Feature Relationships (Scatter Plot Matrix)",
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 20}
    },
    height=600,
    width=1000,
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    plot_bgcolor='white'
)

fig_scatter.show()

# 2. FEATURE DISTRIBUTIONS (HISTOGRAMS)
print("Creating Feature Distribution Plots...")

fig_hist = make_subplots(
    rows=2, 
    cols=2,
    # subplot_titles=feature_names,
    vertical_spacing=0.15,
    horizontal_spacing=0.15
)

for idx, (col_name, feature_name) in enumerate(zip(feature_cols, feature_names)):
    row = (idx // 2) + 1
    col = (idx % 2) + 1
    
    for class_name, color in color_discrete_map.items():
        class_data = df[df['class'] == class_name]
        
        fig_hist.add_trace(
            go.Histogram(
                x=class_data[col_name],
                name=class_name,
                marker_color=color,
                opacity=0.7,
                nbinsx=15,
                showlegend=(idx == 0),  # Only show legend once
                legendgroup=class_name
            ),
            row=row, col=col
        )
    
    # Update axes for this subplot
    fig_hist.update_xaxes(
        title_text=feature_name, 
        row=row, col=col,
        showgrid=True,
        gridcolor='lightgray'
    )
    fig_hist.update_yaxes(
        title_text="Count", 
        row=row, col=col,
        showgrid=True,
        gridcolor='lightgray'
    )

# Update histogram layout
fig_hist.update_layout(
    title={
        'text': "Iris Dataset - Feature Distributions",
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 20}
    },
    height=600,
    width=800,
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    plot_bgcolor='white',
    barmode='overlay'  # Overlay histograms for better comparison
)

fig_hist.show()

Creating Scatter Plot Matrix...


Creating Feature Distribution Plots...


# How PCA works?

PCA uses linear algebra to transform data into new features called principal components. It finds these by calculating eigenvectors (directions) and eigenvalues (importance) from the covariance matrix. PCA selects the top components with the highest eigenvalues and projects the data onto them simplify the dataset.



## Step 1: Standardize the Data

Different features may have different units and scales like salary vs. age. To compare them fairly PCA first standardizes the data by making each feature have:

* A mean of 0
* A standard deviation of 1

$Z = \frac{X - \mu}{\sigma}$

where:

* μ is the mean of independent features μ ={μ1,μ2,⋯,μm}
* σ is the standard deviation of independent features σ ={σ1,σ2,⋯,σm}

In [19]:
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

In [20]:
print("Before Standardization:")
print(X[:5])
print("After Standardization:")
print(X_std[:5])

Before Standardization:
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
After Standardization:
[[-0.90068117  1.03205722 -1.3412724  -1.31297673]
 [-1.14301691 -0.1249576  -1.3412724  -1.31297673]
 [-1.38535265  0.33784833 -1.39813811 -1.31297673]
 [-1.50652052  0.10644536 -1.2844067  -1.31297673]
 [-1.02184904  1.26346019 -1.3412724  -1.31297673]]


In [23]:
print("The mean of each feature of the X dataset is:")
print(scaler.mean_)
print("The standard deviation of each feature of the X dataset is:")
print(scaler.scale_)


The mean of each feature of the X dataset is:
[5.84333333 3.054      3.75866667 1.19866667]
The standard deviation of each feature of the X dataset is:
[0.82530129 0.43214658 1.75852918 0.76061262]


## Step 2: Calculate Covariance Matrix

Next PCA calculates the covariance matrix to see how features relate to each other whether they increase or decrease together. The covariance between two features x1 and x2 is:

$ cov(x_1, x_2) = \frac{\sum_{i=1}^{n} (x1_i - \tilde{x1})(x2_i - \tilde{x2})}{n-1} $

Where:

* $\tilde{x1}$ and $\tilde{x2}$ are the mean values of features x1 and x2
* n is the number of data points

The value of covariance can be positive, negative or zeros.

In [29]:
mean_vec = np.mean(X_std, axis=0)
cov_mat = (X_std - mean_vec).T.dot((X_std - mean_vec)) / (X_std.shape[0]-1)
print(f'Covariance matrix \n{cov_mat}')

Covariance matrix 
[[ 1.00671141 -0.11010327  0.87760486  0.82344326]
 [-0.11010327  1.00671141 -0.42333835 -0.358937  ]
 [ 0.87760486 -0.42333835  1.00671141  0.96921855]
 [ 0.82344326 -0.358937    0.96921855  1.00671141]]


In [30]:
print(f'NumPy covariance matrix: \n{np.cov(X_std.T)}')

NumPy covariance matrix: 
[[ 1.00671141 -0.11010327  0.87760486  0.82344326]
 [-0.11010327  1.00671141 -0.42333835 -0.358937  ]
 [ 0.87760486 -0.42333835  1.00671141  0.96921855]
 [ 0.82344326 -0.358937    0.96921855  1.00671141]]
